We are going to complexify the code a little so as to get closer to the theoretical model. 

We still have 2 locations, firms, workers, a labor market and a housing (housing means any building, so it can be for firms or workers) market in each location, with commuting flows. 

We still consider only 1 type of labor (there are 3 types in the model) D. 

The labor demand from the firm in location j depends on wage w_j (as in the current code), but also on the price of housing p_j.\
The housing demand from the firm also depends on both p_j and w_j.  

* 2 locations
* 1 firm in each location 
* 2 workers in each location 

parameters : 
$\alpha, \delta, \eta_1, \eta_2, \Psi_1, \Psi_2$

- w stands for a specific worker
- j stands for a specific firm
- m is the firm location
- $\Psi$ is the firm TFP (total factor productivity)

Labor demand in m = 1, 2 
$
D_1 = \bigg(\frac{\big(\Psi_1\beta p^{\alpha-1}_{1}\big)^{(1/\alpha)}}{w_1} \bigg)^{1/(1-\delta)}\\
D_2 = \bigg(\frac{\big(\Psi_2\beta p^{\alpha-1}_{2}\big)^\frac{1}{\alpha}}{w_2}\bigg)^\frac{1}{1-\delta}
$


Commutes between 1 & 2 
$
M_{12} = max(Pop_1\frac{w_2 - w_1}{w_1} , 0)\\
M_{21} = max(Pop_2\frac{w_1 - w_2}{w_2} , 0)
$


Labor supply
$
LS_1 = Pop_1 - M_{12} + M_{21}\\
LS_2 = Pop_2 - M_{21} + M_{12}
$


Housing demand of firms 
$
H_1 = \frac{1-\alpha}{\alpha}\frac{w_1D_1}{p_{1}}\\
H_2 = \frac{1-\alpha}{\alpha}\frac{w_2D_2}{p_{2}}
$


Housing demand of households  
$
Hh_1 = 0.3Pop_1\frac{w_1}{p_1}\\
Hh_2 = 0.3Pop_2\frac{w_2}{p_2}
$


Housing supply 
$
HT_1 = 0.2 + δ_1p_1 + δ_2p_1^2\\
HT_2 = 0.2 + δ_1p_2 + δ_2p_2^2
$


$$
Loss_1 = (LS_1 - D_1)^2 \\
Loss_2 = (LS_2 - D_2)^2\\
Loss_3 = (H_1 + Hh_1 - HT_1)^2\\
Loss_4 = (H_2 + Hh_2 - HT_2)^2
$$

In [1]:
using Pkg, Optim, Ipopt, JuMP, Test, Printf, Plots, EAGO

# Functions

## intermediate_model_loss

In [2]:
# Function intermediate_model_loss
# finds the Loss value
# for given values w_1, w_2, p_1, p_2, Pop_1, Pop_2, Ψ_1, Ψ_2, α, δ, η_1, η_2

function intermediate_model_loss(w_1, w_2, p_1, p_2, Pop_1, Pop_2, Ψ_1, Ψ_2, α, δ, η_1, η_2)
    β = α^α * (1-α)^(1-α)
    
    #---------------------------------------------------------------------------------------
    # FUNCTIONS
    #---------------------------------------------------------------------------------------
    ## Labor demand
    temp1 = 1/(1-δ)
    temp2 = α-1
    temp2_1 = p_1^temp2
    temp2_2 = p_2^temp2
    temp3 = 1/α
    temp4_1 = Ψ_1 * β
    temp4_2 = Ψ_2 * β
    temp_1 = temp4_1 * temp2_1
    temp_2 = temp4_2 * temp2_2
    temp_1 = temp_1 ^ temp3
    temp_2 = temp_1 ^ temp3
    temp_1 = temp_1 / w_1
    temp_2 = temp_2 / w_2
    D_1 = temp_1 ^ temp1
    D_2 = temp_2 ^ temp1
    
    D_1 = (((Ψ_1 * β * p_1^(α-1)) ^ (1/α)) / w_1) ^ (1/(1-δ))
    D_2 = (((Ψ_2 * β * p_2^(α-1)) ^ (1/α)) / w_2) ^ (1/(1-δ))
    ## Commutes between 1 & 2
    M_12 = max(0, Pop_1*(w_2 - w_1)/w_1)
    M_21 = max(0, Pop_2*(w_1 - w_2)/w_2)
    ## Labor supply
    LS_1 = Pop_1 - M_12 + M_21
    LS_2 = Pop_2 - M_21 + M_12
    ## Housing demand of firm
    H_1 = - temp2 * temp3 * w_1 * D_1 / p_1
    H_2 = - temp2 * temp3 * w_2 * D_2 / p_2
    ## Housing demand of households
    Hh_1 = 0.3 * Pop_1 * w_1 / p_1
    Hh_2 = 0.3 * Pop_2 * w_2 / p_2
    ## Housing supply
    HT_1 = 0.2 + η_1 * p_1 + η_2 * p_1^2
    HT_2 = 0.2 + η_1 * p_2 + η_2 * p_2^2
    
    #----------------------------------------------------------------------------------------
    # EQUILIBRIUM CONDITIONS
    #----------------------------------------------------------------------------------------
    ## Labor
    Loss_1 = (LS_1 - D_1)^2
    Loss_2 = (LS_2 - D_2)^2
    ## Housing
    Loss_3 = (H_1 + Hh_1 - HT_1)^2
    Loss_4 = (H_2 + Hh_2 -HT_2)^2
    
    Loss = Loss_1 + Loss_2 + Loss_3 + Loss_4
    
    return Loss
    
end

intermediate_model_loss (generic function with 1 method)

In [6]:
function intermediate_model_loss_wrapper(X::Vector{Float64})
    
    intermediate_model_loss(X[1], X[2], X[3], X[4], X[5], X[6], X[7], X[8], X[9], X[10], X[11], X[12])
    
end

intermediate_model_loss_wrapper (generic function with 1 method)

## intermediate_model_loss_solve

In [17]:
# Function intermediate_model_loss_solve
# finds the equilibrium values w_1, w_2, p_1, p_2, Pop_1, Pop_2
# for given values Ψ_1, Ψ_2, α, δ, η_1, η_2

function intermediate_model_loss_solve(o_Ψ_1, o_Ψ_2, o_α, o_δ, o_η_1, o_η_2)
    model = Model(EAGO.Optimizer)        # define empty model solved by EAGO algorithm
    
    register(model, :intermediate_model_loss, 12, intermediate_model_loss ; autodiff = true)
    register(model, :intermediate_model_loss_wrapper, 1, intermediate_model_loss_wrapper, autodiff = true)
    println("on arrive quand même ici.")
    @variable(model, w_1, start = 100.)
    @variable(model, w_2, start = 100.)
    @variable(model, p_1, start = 10.)
    @variable(model, p_2, start = 10.)
    @variable(model, Pop_1, start = 100.)
    @variable(model, Pop_2, start = 200.)
    
    @NLconstraint(model, w_1 >= 0.000001) # w_1 doit être strictement positif
    @NLconstraint(model, w_2 >= 0.000001) # w_2 doit être strictement positif
    @NLconstraint(model, p_1 >= 0.000001) # p_1 doit être strictement positif
    @NLconstraint(model, p_2 >= 0.000001) # p_2 doit être strictement positif
    @NLconstraint(model, Pop_1 >= 0.000001) # Pop_1 doit être strictement positif
    @NLconstraint(model, Pop_2 >= 0.000001) # Pop_2 doit être strictement positif
    
    @NLparameter(model, Ψ_1 == o_Ψ_1)
    @NLparameter(model, Ψ_2 == o_Ψ_2)
    @NLparameter(model, α == o_α)
    @NLparameter(model, δ == o_δ)
    @NLparameter(model, η_1 == o_η_1)
    @NLparameter(model, η_2 == o_η_2)
    
    @NLobjective(model, Min, intermediate_model_loss_wrapper([w_1, w_2, p_1, p_2, Pop_1, Pop_2,
                 Ψ_1, Ψ_2, α, δ, η_1, η_2]))
    
    JuMP.optimize!(model)
    
    return (JuMP.value(w_1), JuMP.value(w_2), JuMP.value(p_1), JuMP.value(p_2),
            JuMP.value(Pop_1), JuMP.value(Pop_2))
    
end

intermediate_model_loss_solve (generic function with 1 method)

In [18]:
intermediate_model_loss_solve(1, 1, 0.5, 0.5, 0.1, 0.01)

LoadError: Unable to register the function :intermediate_model_loss_wrapper.

Common reasons for this include:

 * The function takes `f(x::Vector)` as input, instead of the splatted
   `f(x...)`.
 * The function assumes `Float64` will be passed as input, it must work for any
   generic `Real` type.
 * The function allocates temporary storage using `zeros(3)` or similar. This
   defaults to `Float64`, so use `zeros(T, 3)` instead.

As examples, instead of:
```julia
my_function(x::Vector) = sum(x.^2)
```
use:
```julia
my_function(x::T...) where {T<:Real} = sum(x[i]^2 for i in 1:length(x))
```

Instead of:
```julia
function my_function(x::Float64...)
    y = zeros(length(x))
    for i in 1:length(x)
        y[i] = x[i]^2
    end
    return sum(y)
end
```
use:
```julia
function my_function(x::T...) where {T<:Real}
    y = zeros(T, length(x))
    for i in 1:length(x)
        y[i] = x[i]^2
    end
    return sum(y)
end
```

Review the stacktrace below for more information, but it can often 

In [5]:
intermediate_model_loss_solve(10, 20, 0.5, 0.5, 0.8, 0.01)

LoadError: BoundsError: attempt to access 7-element Vector{MC{7, NS}} at index [1:12]

In [18]:
# Function intermediate_model_loss
# finds the Loss value
# for given values w_1, w_2, p_1, p_2, Pop_1, Pop_2, Ψ_1, Ψ_2, α, δ, η_1, η_2

function intermediate_model_loss(w_1, w_2, p_1, p_2, Pop_1, Pop_2)
    β = α^α * (1-α)^(1-α)
    
    #---------------------------------------------------------------------------------------
    # FUNCTIONS
    #---------------------------------------------------------------------------------------
    ## Labor demand
    D_1 = 1/(32*p_1^2*w_1^2)
    D_2 = 1/(32*p_2^2*w_2^2)
    ## Commutes between 1 & 2
    M_12 = max(0, Pop_1*(w_2 - w_1)/w_1)
    M_21 = max(0, Pop_2*(w_1 - w_2)/w_2)
    ## Labor supply
    LS_1 = Pop_1 - M_12 + M_21
    LS_2 = Pop_2 - M_21 + M_12
    ## Housing demand of firm
    H_1 = w_1 * D_1 / p_1
    H_2 = w_2 * D_2 / p_2
    ## Housing demand of households
    Hh_1 = 0.3 * Pop_1 * w_1 / p_1
    Hh_2 = 0.3 * Pop_2 * w_2 / p_2
    ## Housing supply
    HT_1 = 0.2 + 0.2 * p_1
    HT_2 = 0.2 + 0.2 * p_2
    
    #----------------------------------------------------------------------------------------
    # EQUILIBRIUM CONDITIONS
    #----------------------------------------------------------------------------------------
    ## Labor
    Loss_1 = (LS_1 - D_1)^2
    Loss_2 = (LS_2 - D_2)^2
    ## Housing
    Loss_3 = (H_1 + Hh_1 - HT_1)^2
    Loss_4 = (H_2 + Hh_2 -HT_2)^2
    
    Loss = Loss_1 + Loss_2 + Loss_3 + Loss_4
    
    return Loss
    
end

intermediate_model_loss (generic function with 3 methods)

In [3]:
# Function intermediate_model_loss_solve
# finds the equilibrium values w_1, w_2, p_1, p_2, Pop_1, Pop_2
# for given values Ψ_1, Ψ_2, α, δ, η_1, η_2

function intermediate_model_loss_solve()
    model = Model(Ipopt.Optimizer)        # define empty model solved by Ipopt algorithm
    
    register(model, :intermediate_model_loss, 6, intermediate_model_loss ; autodiff = true)
    
    @variable(model, w_1, start = 100.)
    @variable(model, w_2, start = 100.)
    @variable(model, p_1, start = 10.)
    @variable(model, p_2, start = 10.)
    @variable(model, Pop_1, start = 100.)
    @variable(model, Pop_2, start = 200.)
    
    @NLconstraint(model, w_1 >= 0)
    @NLconstraint(model, w_2 >= 0)
    @NLconstraint(model, p_1 >= 0)
    @NLconstraint(model, p_2 >= 0)
    @NLconstraint(model, Pop_1 >= 0)
    @NLconstraint(model, Pop_2 >= 0)
    
    @NLobjective(model, Min, intermediate_model_loss(w_1, w_2, p_1, p_2, Pop_1, Pop_2))
    
    JuMP.optimize!(model)
    
    return (JuMP.value(w_1), JuMP.value(w_2), JuMP.value(p_1), JuMP.value(p_2),
            JuMP.value(Pop_1), JuMP.value(Pop_2))
    
end

intermediate_model_loss_solve (generic function with 1 method)

In [16]:
intermediate_model_loss_solve()

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.5.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        6
        inequality constraints with only lower bounds:        6
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.9604968e+05 0.00e+00 5.05e+01   0.0 0.00e+00    -  0.00e+00 0.00e+00 

(3200.868956702498, 2779.8172561648953, 4.060355482443083, 3.655003846550446, 0.0042793420493154365, 0.004080310446830555)

In [2]:
# Function intermediate_model_loss
# finds the Loss value
# for given values w_1, w_2, p_1, p_2, Pop_1, Pop_2, Ψ_1, Ψ_2, α, δ, η_1, η_2

function intermediate_model_loss(w_1::Real, w_2::Real, p_1::Real, p_2::Real, Pop_1::Real, Pop_2::Real)
    β = α^α * (1-α)^(1-α)
    
    #---------------------------------------------------------------------------------------
    # FUNCTIONS
    #---------------------------------------------------------------------------------------
    ## Labor demand
    D_1 = convert(Float64, Complex(((Ψ_1 * β * p_1^(α-1)) ^(1/α)) / w_1) ^ (1/(1-δ)))
    D_2 = convert(Float64, Complex(((Ψ_2 * β * p_2^(α-1)) ^(1/α)) / w_2) ^ (1/(1-δ)))
    ## Commutes between 1 & 2
    M_12 = max(0, Pop_1*(w_2 - w_1)/w_1)
    M_21 = max(0, Pop_2*(w_1 - w_2)/w_2)
    ## Labor supply
    LS_1 = Pop_1 - M_12 + M_21
    LS_2 = Pop_2 - M_21 + M_12
    ## Housing demand of firm
    H_1 = ((1-α)/α) * w_1 * D_1 / p_1
    H_2 = ((1-α)/α) * w_2 * D_2 / p_2
    ## Housing demand of households
    Hh_1 = 0.3 * Pop_1 * w_1 / p_1
    Hh_2 = 0.3 * Pop_2 * w_2 / p_2
    ## Housing supply
    HT_1 = 0.2 + η_1 * p_1 + η_2 * p_1^2
    HT_2 = 0.2 + η_1 * p_2 + η_2 * p_2^2
    
    #----------------------------------------------------------------------------------------
    # EQUILIBRIUM CONDITIONS
    #----------------------------------------------------------------------------------------
    ## Labor
    Loss_1 = (LS_1 - D_1)^2
    Loss_2 = (LS_2 - D_2)^2
    ## Housing
    Loss_3 = (H_1 + Hh_1 - HT_1)^2
    Loss_4 = (H_2 + Hh_2 -HT_2)^2
    
    Loss = Loss_1 + Loss_2 + Loss_3 + Loss_4
    
    return Loss
    
end

intermediate_model_loss (generic function with 1 method)

In [4]:
Ψ_1, Ψ_2, α, δ, η_1, η_2 = 1., 1., 1/4, 1/4, 0.2, 0
intermediate_model_loss_solve()

LoadError: Unable to register the function :intermediate_model_loss.

Common reasons for this include:

 * The function takes `f(x::Vector)` as input, instead of the splatted
   `f(x...)`.
 * The function assumes `Float64` will be passed as input, it must work for any
   generic `Real` type.
 * The function allocates temporary storage using `zeros(3)` or similar. This
   defaults to `Float64`, so use `zeros(T, 3)` instead.

As examples, instead of:
```julia
my_function(x::Vector) = sum(x.^2)
```
use:
```julia
my_function(x::T...) where {T<:Real} = sum(x[i]^2 for i in 1:length(x))
```

Instead of:
```julia
function my_function(x::Float64...)
    y = zeros(length(x))
    for i in 1:length(x)
        y[i] = x[i]^2
    end
    return sum(y)
end
```
use:
```julia
function my_function(x::T...) where {T<:Real}
    y = zeros(T, length(x))
    for i in 1:length(x)
        y[i] = x[i]^2
    end
    return sum(y)
end
```

Review the stacktrace below for more information, but it can often be hard 

## intermediate_model_loss_solve_2

In [21]:
# Function intermediate_model_loss_solve_2
# finds equilibrium values Ψ_1, Ψ_2, α, δ, η_1, η_2
# for given values w_1, w_2, p_1, p_2, Pop_1, Pop_2

function intermediate_model_loss_solve_2(a_w_1, a_w_2, a_p_1, a_p_2, a_Pop_1, a_Pop_2)
    
    model = Model(Ipopt.Optimizer)
    
    register(model, :intermediate_model_loss, 12, intermediate_model_loss ; autodiff = true)
    
    @variable(model, Ψ_1, start = 1.)
    @variable(model, Ψ_2, start = 1.)
    @variable(model, α, start = 1/2)
    @variable(model, δ, start = 1/2)
    @variable(model, η_1, start = 0.2)
    @variable(model, η_2, start = 0.)
    
    @NLconstraint(model, Ψ_1 >= 0)
    @NLconstraint(model, Ψ_2 >= 0)
    @NLconstraint(model, 0.0001 <= α <= 1)
    @NLconstraint(model, 0 <= δ <= 0.99999)
    
    @NLparameter(model, w_1 == a_w_1)
    @NLparameter(model, w_2 == a_w_2)
    @NLparameter(model, p_1 == a_p_1)
    @NLparameter(model, p_2 == a_p_2)
    @NLparameter(model, Pop_1 == a_Pop_1)
    @NLparameter(model, Pop_2 == a_Pop_2)
    
    @NLobjective(model, Min, intermediate_model_loss(w_1, w_2, p_1, p_2, Pop_1, Pop_2, Ψ_1, Ψ_2, α, δ, η_1, η_2))
    
    JuMP.optimize!(model)
    
    println("RESULTS:")
    println(" $(Ψ_1) = $(JuMP.value(Ψ_1)) ")
    println(" $(Ψ_2) = $(JuMP.value(Ψ_2)) ")
    println(" $(α) = $(JuMP.value(α)) ")
    println(" $(δ) = $(JuMP.value(δ)) ")
    println(" $(η_1) = $(JuMP.value(η_1)) ")
    println(" $(η_2) = $(JuMP.value(η_2)) ")

    return(JuMP.value(Ψ_1), JuMP.value(Ψ_2), JuMP.value(α), JuMP.value(δ), JuMP.value(η_1), JuMP.value(η_2))   
    
end

intermediate_model_loss_solve_2 (generic function with 1 method)

In [22]:
intermediate_model_loss_solve_2(100, 100, 30, 30, 100, 100)

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.5.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        4
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        4
        inequality constraints with only lower bounds:        2
   inequality constraints with lower and upper bounds:        2
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  3.7596880e+04 0.00e+00 1.00e+02   0.0 0.00e+00    -  0.00e+00 0.00e+00 

LoadError: DomainError with -9.999972627738885e-9:
Exponentiation yielding a complex result requires a complex argument.
Replace x^y with (x+0im)^y, Complex(x)^y, or similar.

## intermediate_model_outer_loss

In [10]:
# Function intermediate_model_outer_loss
# finds the Outer_Loss value
# for given values w_1, w_2, p_1, p_2, Pop_1, Pop_2

function intermediate_model_outer_loss(o_w_1, o_w_2, o_p_1, o_p_2, o_Pop_1, o_Pop_2)
    
    Outer_Loss_w_1 = (o_w_1 - w_1_target)^2
    Outer_Loss_w_2 = (o_w_2 - w_2_target)^2
    Outer_Loss_p_1 = (o_p_1 - p_1_target)^2
    Outer_Loss_p_2 = (o_p_2 - p_2_target)^2
    Outer_Loss_Pop_1 = (o_Pop_1 - Pop_1_target)^2
    Outer_Loss_Pop_2 = (o_Pop_2 - Pop_2_target)^2
    
    Outer_Loss = (Outer_Loss_w_1 + Outer_Loss_w_2 + Outer_Loss_p_1 + Outer_Loss_p_2 +
                  Outer_Loss_Pop_1 + Outer_Loss_Pop_2)
    
    return Outer_Loss
    
end

intermediate_model_outer_loss (generic function with 1 method)

# Tests

## Tests de la fonction intermediate_model_loss

In [21]:
intermediate_model_loss(150, 200, 30, 70, 90, 200, 10, 20, 0.5, 0.5, 1, 0.01)

68405.48129531654

In [25]:
intermediate_model_loss(100., 50., 10., 10., 20., 25., 3., 3., 0.5, -0.5, 0., 0.) 

7021.536693449352

In [26]:
intermediate_model_loss(80, 200, 30, 70, 80, 200, 1, 1, 0.5, 0.5, 0.2, 0)

132061.66367776477

## Tests de la fonction intermediate_model_loss_solve

In [34]:
intermediate_model_loss_solve(10, 20, 0.5, 0.5, 0.1, 0.01)

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.5.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        6
        inequality constraints with only lower bounds:        6
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.9616885e+05 0.00e+00 5.05e+01   0.0 0.00e+00    -  0.00e+00 0.00e+00 

(40.72461757008275, 50.96606586365038, 3.2470560955403034, 5.987498666010296, 0.04836944123202679, 0.09532415668520396)

In [25]:
# Function intermediate_model_solve_2 finds equilibrium values of parameters Ψ_1, Ψ_2, α, δ, θ_1, θ_2, η_1, η_2
# for given values w_1, w_2, p_1, p_2, Pop_1, Pop_2 

function intermediate_model_solve_2(aw_1, aw_2, ap_1, ap_2, aPop_1, aPop_2)
    
    @printf "Begin solver execution\n"

    @printf "Initializing model in solver\n"
    model = Model(Ipopt.Optimizer)        # define empty model solved by Ipopt algorithm
    #set_attribute(model, "print_level", 8)

    register(model, :intermediate_model_loss, 14, intermediate_model_loss; autodiff = true)

    @NLparameter(model, w_1 == aw_1)
    @NLparameter(model, w_2 == aw_2)
    @NLparameter(model, p_1 == ap_1)
    @NLparameter(model, p_2 == ap_2)
    @NLparameter(model, Pop_1 == aPop_1)
    @NLparameter(model, Pop_2 == aPop_2)
    
    #@NLparameter(model, Ψ_1 == 1.)
    @variable(model, Ψ_1 >= 0, start = 1.)
    #@NLparameter(model, Ψ_2 == 1.)
    @variable(model, Ψ_2 >= 0, start = 1.)
    #@NLparameter(model, α == 0.5)
    @variable(model, 0 <= α <= 1, start = 0.1)
    #@NLparameter(model, δ == 0.)
    @variable(model, δ, start = 0.2)
    #@NLparameter(model, θ_1 == 0.2)
    @variable(model, θ_1 >= 0, start = 0.1)
    #@NLparameter(model, θ_2 == 0.2)
    @variable(model, θ_2 >= 0, start = 0.1)
    #@NLparameter(model, η_1 == 0.2)
    @variable(model, η_1, start = 0.2)
    #@NLparameter(model, η_2 == 0.2)
    @variable(model, η_2, start = 0.)
    
    #@NLconstraint(model, 0 <= α <= 1)
    #@NLconstraint(model, Ψ_1 >= 0)
    #@NLconstraint(model, Ψ_2 >= 0)
    #@NLconstraint(model, θ_1 >= 0)
    #@NLconstraint(model, θ_2 >= 0)

    # next, we set an objective function
    @NLobjective(model, Min, intermediate_model_loss(w_1, w_2, p_1, p_2, Pop_1, Pop_2, Ψ_1, Ψ_2, α, δ, θ_1, θ_2, η_1, η_2))

    @printf "Model initialized\n"

    @printf "Launching optimization\n"
    JuMP.optimize!(model)
    @printf "Optimization complete\n"
    @printf "Results\n"

    println("RESULTS:")
    println(" $(Ψ_1) = $(JuMP.value(Ψ_1)) ")
    println(" $(Ψ_2) = $(JuMP.value(Ψ_2)) ")
    println(" $(α) = $(JuMP.value(α)) ")
    println(" $(δ) = $(JuMP.value(δ)) ")
    println(" $(θ_1) = $(JuMP.value(θ_1)) ")
    println(" $(θ_2) = $(JuMP.value(θ_2)) ")
    println(" $(η_1) = $(JuMP.value(η_1)) ")
    println(" $(η_2) = $(JuMP.value(η_2)) ")

    return(JuMP.value(Ψ_1), JuMP.value(Ψ_2), JuMP.value(α), JuMP.value(δ), JuMP.value(θ_1), JuMP.value(θ_2), JuMP.value(η_1), JuMP.value(η_2))
    
end

    #set_silent(model)
    
    # @NLconstraint(model, α != 0.)
    # @NLconstraint(model, δ != 1.)
    # @NLconstraint(model, θ_1 != 0.)
    # @NLconstraint(model, θ_2 != 0.)

intermediate_model_solve_2 (generic function with 1 method)

In [26]:
intermediate_model_solve_2(150, 100, 30, 70, 90, 100)

Begin solver execution
Initializing model in solver
Model initialized
Launching optimization
This is Ipopt version 3.14.10, running with linear solver MUMPS 5.5.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        8
                     variables with only lower bounds:        4
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_

(1.0000138446073217, 1.0000138446073217, 0.10006644437220243, 0.2, 0.10007602244749736, 0.10007602244749736, 7.406292517073779, -0.0970986394566318)

In [12]:
intermediate_model_solve_2(100, 100, 30, 30, 150, 150)

Begin solver execution
Initializing model in solver
Model initialized
Launching optimization
This is Ipopt version 3.14.10, running with linear solver MUMPS 5.5.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        8
                     variables with only lower bounds:        4
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_

LoadError: DomainError with -3.376186630216541e-10:
Exponentiation yielding a complex result requires a complex argument.
Replace x^y with (x+0im)^y, Complex(x)^y, or similar.

In [55]:
# Function toy_model_outer_solve the model using moments
# = finds parameters values which minimize the gap between observed moments and model moments,
# that is equilibrium values of w_1, w_2, p_1, p_2, Pop_1, Pop_2

function intermediate_model_outer_solve()   
    
    #Ψ_1_start, Ψ_2_start, α_start, δ_start, θ_1_start, θ_2_start, η_1_start, η_2_start = intermediate_model_solve_2(w_1_target, w_2_target, p_1_target, p_2_target, Pop_1_target, Pop_2_target)
    Ψ_1_start, Ψ_2_start, α_start, δ_start, θ_1_start, θ_2_start, η_1_start, η_2_start = 1, 1, 0.5, 0.5, 1, 1, 0.2, 0
    
    println(" Ψ_1_init = $(Ψ_1_start) ")
    println(" Ψ_2_init = $(Ψ_2_start) ")
    println(" α_init = $(α_start) ")
    println(" δ_init = $(δ_start) ")
    println(" θ_1_init = $(θ_1_start) ")
    println(" θ_2_init = $(θ_2_start) ")
    println(" η_1_init = $(η_1_start) ")
    println(" η_2_init = $(η_2_start) ")
    
    Outer_intermediate = Model(Ipopt.Optimizer)            # define empty model solved by Ipopt algorithm
    set_attribute(Outer_intermediate, "print_level", 5)

    register(Outer_intermediate, :intermediate_model_pop_outer, 6, intermediate_model_pop_outer; autodiff = true)
    println("Model initialized\n")

    @variable(Outer_intermediate, Ψ_1 >= 0, start = Ψ_1_start)
    @variable(Outer_intermediate, Ψ_2 >= 0, start = Ψ_2_start)
    @variable(Outer_intermediate, 0 <= α <= 1, start = α_start)
    @variable(Outer_intermediate, 0 <= δ <= 1, start = δ_start)
    @variable(Outer_intermediate, θ_1 >= 0, start = θ_1_start)
    @variable(Outer_intermediate, θ_2 >= 0, start = θ_2_start)
    @NLparameter(Outer_intermediate, η_1 == 0.2)
    @NLparameter(Outer_intermediate, η_2 == -0.05)
    #@variable(Outer_intermediate, η_1, start = η_1_start)
    #@variable(Outer_intermediate, η_2, start = η_2_start)

    @variable(Outer_intermediate, w_1 >= 0, start = w_1_target)
    @variable(Outer_intermediate, w_2 >= 0, start = w_2_target)
    @variable(Outer_intermediate, p_1 >= 0, start = p_1_target)
    @variable(Outer_intermediate, p_2 >= 0, start = p_2_target)
    @variable(Outer_intermediate, Pop_1 >= 0, start = Pop_1_target)
    @variable(Outer_intermediate, Pop_2 >= 0, start = Pop_2_target)
    
    @NLconstraint(Outer_intermediate, (Pop_1 - (max( Pop_1 * (w_2 - w_1)/w_1 , 0)) + (max( Pop_2 * (w_1 - w_2)/w_2 , 0)) - (θ_1*(Ψ_1*α^α*(1-α)^(1-α)*p_1^(α-1))^(1/α)/w_1)^(1/(1-δ)))^2 == 0) # Loss_1 ~= 0
    @NLconstraint(Outer_intermediate, (Pop_2 - (max( Pop_2 * (w_1 - w_2)/w_2 , 0)) + (max( Pop_1 * (w_2 - w_1)/w_1 , 0)) - (θ_2*(Ψ_2*α^α*(1-α)^(1-α)*p_2^(α-1))^(1/α)/w_2)^(1/(1-δ)))^2 == 0)# Loss_2 ~= 0
    @NLconstraint(Outer_intermediate, ((1-α) * w_1 * ((θ_1*(Ψ_1*α^α*(1-α)^(1-α)*p_1^(α-1))^(1/α)/w_1)^(1/(1-δ))) / (α * p_1) + Pop_1 * 0.3 * w_1 / p_1 - (0.2 + η_1 * p_1 + η_2 * p_1^2))^2 == 0) # Loss_3 ~= 0
    @NLconstraint(Outer_intermediate, ((1-α) * w_2 * ((θ_2*(Ψ_2*α^α*(1-α)^(1-α)*p_2^(α-1))^(1/α)/w_2)^(1/(1-δ))) / (α * p_2) + Pop_2 * 0.3 * w_2 / p_2 - (0.2 + η_1 * p_2 + η_2 * p_2^2))^2 == 0) # Loss_4 ~= 0
    
    
    # next, we set an objective function
    @NLobjective(Outer_intermediate, Min, intermediate_model_pop_outer(w_1, w_2, p_1, p_2, Pop_1, Pop_2))
       
    #At this stage JuMP has a mathematical representation of our model internalized
    JuMP.optimize!(Outer_intermediate)
    
    println("RESULTS:")
    println("Outer_Loss = $(JuMP.objective_value(Outer_intermediate))")
    println()
    println(" $(w_1) = $(JuMP.value(w_1)) ")
    println(" $(w_2) = $(JuMP.value(w_2)) ")
    println(" $(p_1) = $(JuMP.value(p_1)) ")
    println(" $(p_2) = $(JuMP.value(p_2)) ")
    println(" $(Pop_1) = $(JuMP.value(Pop_1)) ")
    println(" $(Pop_2) = $(JuMP.value(Pop_2)) ")
    println()
    println(" $(Ψ_1) = $(JuMP.value(Ψ_1)) ")
    println(" $(Ψ_2) = $(JuMP.value(Ψ_2)) ")
    println(" $(α) = $(JuMP.value(α)) ")
    println(" $(δ) = $(JuMP.value(δ)) ")
    println(" $(θ_1) = $(JuMP.value(θ_1)) ")
    println(" $(θ_2) = $(JuMP.value(θ_2)) ")
    println(" $(η_1) = $(JuMP.value(η_1)) ")
    println(" $(η_2) = $(JuMP.value(η_2)) ")
    
    return(JuMP.value(Ψ_1), JuMP.value(Ψ_2), JuMP.value(α), JuMP.value(δ), JuMP.value(θ_1), JuMP.value(θ_2), JuMP.value(η_1), JuMP.value(η_2))
end           

intermediate_model_outer_solve (generic function with 1 method)

# Tests

## Test 1

In [56]:
w_1_target=150
w_2_target=200
p_1_target=30
p_2_target=70
Pop_1_target=90
Pop_2_target=200

200

In [57]:
Ψ_1, Ψ_2 , α, δ, θ_1, θ_2, η_1, η_2 = intermediate_model_outer_solve()
intermediate_model_loss(w_1_target, w_2_target, p_1_target, p_2_target, Pop_1_target, Pop_2_target, Ψ_1, Ψ_2 , α, δ, θ_1, θ_2, η_1, η_2)

 Ψ_1_init = 1 
 Ψ_2_init = 1 
 α_init = 0.5 
 δ_init = 0.5 
 θ_1_init = 1 
 θ_2_init = 1 
 η_1_init = 0.2 
 η_2_init = 0 
Model initialized

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.5.1.

Number of nonzeros in equality constraint Jacobian...:       32
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       12
                     variables with only lower bounds:       10
                variables with lower and upper bounds:        2
                     variables with only upper bounds:        0
Total number of equality constraints.................:        4
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective

248494.26320069024

## Test 2

In [49]:
w_1_target=150
w_2_target=100
p_1_target=100
p_2_target=70
Pop_1_target=80
Pop_2_target=200

Ψ_1, Ψ_2 , α, δ, θ_1, θ_2, η_1, η_2 = intermediate_model_outer_solve()
intermediate_model_loss(w_1_target, w_2_target, p_1_target, p_2_target, Pop_1_target, Pop_2_target, Ψ_1, Ψ_2 , α, δ, θ_1, θ_2, η_1, η_2)

 Ψ_1_init = 1 
 Ψ_2_init = 1 
 α_init = 0.5 
 δ_init = 0.5 
 θ_1_init = 1 
 θ_2_init = 1 
 η_1_init = 0.2 
 η_2_init = 0 
Model initialized

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.5.1.

Number of nonzeros in equality constraint Jacobian...:       36
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       14
                     variables with only lower bounds:       10
                variables with lower and upper bounds:        2
                     variables with only upper bounds:        0
Total number of equality constraints.................:        4
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective

3.1807817954879676e-12

## test 3

In [50]:
w_1_target=70
w_2_target=200
p_1_target=30
p_2_target=130
Pop_1_target=100
Pop_2_target=150

150

In [51]:
Ψ_1, Ψ_2 , α, δ, θ_1, θ_2, η_1, η_2 = intermediate_model_outer_solve()
intermediate_model_loss(w_1_target, w_2_target, p_1_target, p_2_target, Pop_1_target, Pop_2_target, Ψ_1, Ψ_2 , α, δ, θ_1, θ_2, η_1, η_2)

 Ψ_1_init = 1 
 Ψ_2_init = 1 
 α_init = 0.5 
 δ_init = 0.5 
 θ_1_init = 1 
 θ_2_init = 1 
 η_1_init = 0.2 
 η_2_init = 0 
Model initialized

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.5.1.

Number of nonzeros in equality constraint Jacobian...:       36
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       14
                     variables with only lower bounds:       10
                variables with lower and upper bounds:        2
                     variables with only upper bounds:        0
Total number of equality constraints.................:        4
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective

129743.91901014655

In [52]:
w_1_target=70
w_2_target=200
p_1_target=30
p_2_target=130
Pop_1_target=100
Pop_2_target=80

80

In [53]:
Ψ_1, Ψ_2 , α, δ, θ_1, θ_2, η_1, η_2 = intermediate_model_outer_solve()
intermediate_model_loss(w_1_target, w_2_target, p_1_target, p_2_target, Pop_1_target, Pop_2_target, Ψ_1, Ψ_2 , α, δ, θ_1, θ_2, η_1, η_2)

 Ψ_1_init = 1 
 Ψ_2_init = 1 
 α_init = 0.5 
 δ_init = 0.5 
 θ_1_init = 1 
 θ_2_init = 1 
 η_1_init = 0.2 
 η_2_init = 0 
Model initialized

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.5.1.

Number of nonzeros in equality constraint Jacobian...:       36
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       14
                     variables with only lower bounds:       10
                variables with lower and upper bounds:        2
                     variables with only upper bounds:        0
Total number of equality constraints.................:        4
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective

165301.28490975942

## Test 4

In [33]:
w_1_target=120
w_2_target=80
p_1_target=60
p_2_target=40
Pop_1_target=100
Pop_2_target=100

100

In [34]:
Ψ_1, Ψ_2 , α, δ, θ_1, θ_2, η_1, η_2 = intermediate_model_outer_solve()
intermediate_model_loss(w_1_target, w_2_target, p_1_target, p_2_target, Pop_1_target, Pop_2_target, Ψ_1, Ψ_2 , α, δ, θ_1, θ_2, η_1, η_2)

 Ψ_1_init = 1 
 Ψ_2_init = 1 
 α_init = 0.5 
 δ_init = -0.5 
 θ_1_init = 1 
 θ_2_init = 1 
 η_1_init = 0.2 
 η_2_init = 0 
Model initialized

Total number of variables............................:       14
                     variables with only lower bounds:       10
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:        4
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

The equality constraints contain an invalid number
The equality constraints contain an invalid number
The equality constraints contain an invalid number
The equality constraints contain an invalid number
The equality constraints contain an invalid number
The equality constraints c

3.9315705307403326e8

## Test 5

In [35]:
w_1_target=110
w_2_target=90
p_1_target=50
p_2_target=60
Pop_1_target=140
Pop_2_target=140

140

In [36]:
Ψ_1, Ψ_2 , α, δ, θ_1, θ_2, η_1, η_2 = intermediate_model_outer_solve()
intermediate_model_loss(w_1_target, w_2_target, p_1_target, p_2_target, Pop_1_target, Pop_2_target, Ψ_1, Ψ_2 , α, δ, θ_1, θ_2, η_1, η_2)

 Ψ_1_init = 1 
 Ψ_2_init = 1 
 α_init = 0.5 
 δ_init = -0.5 
 θ_1_init = 1 
 θ_2_init = 1 
 η_1_init = 0.2 
 η_2_init = 0 
Model initialized

Total number of variables............................:       14
                     variables with only lower bounds:       10
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:        4
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

The equality constraints contain an invalid number
The equality constraints contain an invalid number
The equality constraints contain an invalid number
The equality constraints contain an invalid number
The equality constraints contain an invalid number
The equality constraints c

9.478022956500168e7

## Test 6

In [37]:
w_1_target=11000
w_2_target=9000
p_1_target=5000
p_2_target=6000
Pop_1_target=14000
Pop_2_target=14000

14000

In [38]:
Ψ_1, Ψ_2 , α, δ, θ_1, θ_2, η_1, η_2 = intermediate_model_outer_solve()
intermediate_model_loss(w_1_target, w_2_target, p_1_target, p_2_target, Pop_1_target, Pop_2_target, Ψ_1, Ψ_2 , α, δ, θ_1, θ_2, η_1, η_2)

 Ψ_1_init = 1 
 Ψ_2_init = 1 
 α_init = 0.5 
 δ_init = -0.5 
 θ_1_init = 1 
 θ_2_init = 1 
 η_1_init = 0.2 
 η_2_init = 0 
Model initialized

Total number of variables............................:       14
                     variables with only lower bounds:       10
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:        4
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

The equality constraints contain an invalid number
The equality constraints contain an invalid number
The equality constraints contain an invalid number
The equality constraints contain an invalid number
The equality constraints contain an invalid number
The equality constraints c

5.37891432554885e8

In [8]:
using Pkg, Statistics

In [3]:
mean(abs, [1.0, 3.0, -2.0])

2.0

In [9]:
Pkg.add("Zygote")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed ZygoteRules ───────────────── v0.2.3
   Installed CEnum ─────────────────────── v0.4.2
   Installed RealDot ───────────────────── v0.1.0
   Installed IRTools ───────────────────── v0.4.10
   Installed DataValueInterfaces ───────── v1.0.0
   Installed TableTraits ───────────────── v1.0.1
   Installed AbstractFFTs ──────────────── v1.3.1
   Installed LLVM ──────────────────────── v5.1.0
   Installed GPUArrays ─────────────────── v8.7.1
   Installed Zygote ────────────────────── v0.6.62
   Installed ChainRules ────────────────── v1.50.0
   Installed GPUArraysCore ─────────────── v0.1.5
   Installed LLVMExtra_jll ─────────────── v0.0.21+0
   Installed IteratorInterfaceExtensions ─ v1.0.0
   Installed Tables ────────────────────── v1.10.1
   Installed StructArrays ──────────────── v0.6.15
   Installed PrecompileTools ───────────── v1.1.2
    Updating `~/.julia/environments/v1.8/Project.

In [10]:
using Zygote

p(x) = mean(abs, x)
p'([1.0, 3.0, -2.0])

3-element Vector{Float64}:
  0.3333333333333333
  0.3333333333333333
 -0.3333333333333333